In [5]:
%load_ext autoreload
%autoreload 1
%aimport earl.agents.r2d2.networks
%aimport earl.agents.r2d2.r2d2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import ale_py
import equinox as eqx
import gymnasium
import jax
import jax.numpy as jnp
import jax.random

import earl.agents.r2d2.networks
import earl.agents.r2d2.r2d2 as r2d2

gymnasium.register_envs(ale_py)  # unnecessary but helpful for IDEs

env = gymnasium.make("ALE/Breakout-v5")
env = gymnasium.wrappers.FrameStackObservation(env, stack_size=4)



key = jax.random.PRNGKey(0)
torso_key, lstm_key, dueling_value_key, dueling_advantage_key = jax.random.split(key, num=4)
network = r2d2.R2D2Network(
  torso=earl.agents.r2d2.networks.DeepAtariTorso(
    key=torso_key,
  ),
  lstm_cell=eqx.nn.LSTMCell(512, 512, key=lstm_key),
  dueling_value=eqx.nn.Linear(512, 1, key=dueling_value_key),
  dueling_advantage=eqx.nn.Linear(512, 1, key=dueling_advantage_key),
  num_actions=18,
)

obs = env.observation_space.sample()
print(obs.shape)
obs = jnp.reshape(obs, (obs.shape[0] * obs.shape[1], obs.shape[1], obs.shape[2]))
action = jax.random.randint(key, (1,), 0, 18)
reward = jax.random.uniform(key, (1,))
hidden = (jnp.zeros((1, 512)), jnp.zeros((1, 512)))

print(network(obs, action, reward, hidden))


(4, 210, 160, 3)


ValueError: ('Input to `Conv` needs to have rank 3,', ' but input has shape (4, 210, 160, 3).')